In [1]:
from apify_client import ApifyClient
import matplotlib.pyplot as plt
from skimage import io

import os
from mistralai import Mistral
import json
import functools

In [2]:
def scrap_asos(query, maxItems = 3):
    """ Data scraper from asos.
    Inputs: 
        query (str): search query
        maxItems (int): number of items to scrap
    
    Returns:
        Items (list): list of dictionaries containing the following keys:
            name (str): name of the item
            brandName (str): brand name of the item
            price (str): price of the item
            image (np.array): image of the item
            url (str): url of the item
            gender (str): gender of the item
    """
    
    client = ApifyClient("apify_api_zHwEmmY3hZNab6L57n4NiebpEJDQy42PNRGq")
    
    run_input = {
        "search": query,
        "maxItems": maxItems,
        "endPage": 1,
        "extendOutputFunction": "($) => { return {} }",
        "customMapFunction": "(object) => { return {...object} }",
        "proxy": { "useApifyProxy": True },
    }
    
    run = client.actor("epctex/asos-scraper").call(run_input=run_input)
    
    Items = []
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        try :
            # Certains articles n'ont pas de prix d'où la gestion d'erreur
            price = item["variants"][0]["pricing"]["price"]["current"]["text"]
            
            Items.append({
                "name" : item["name"],
                "brandName" : item["brandName"],
                "price" : price,
                "image" : io.imread(item["images"][0]["url"]),
                "gender" : item["gender"],
                "url" : item["url"]
            })
        except:
            pass
        
    return Items

In [5]:

model = "mistral-large-latest"
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

def query_scrap_asos(query):
    items = scrap_asos(query)
    filtered_items = [{k: v for k, v in item.items() if k != "image"} for item in items]

    return json.dumps(filtered_items, indent=4)

def pipeline_chat_assos(query_content):
    tools = [
        {
            "type": "function",
            "function": {
                "name": "query_scrap_asos",
                "description": "Recommandation d'un article",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "descriptif de l'article désiré",
                        }
                    },
                    "required": ["query"],
                },
            },
        },
    ]


    names_to_functions = {
    'query_scrap_asos': functools.partial(query_scrap_asos)
    }

    messages = [{"role": "user", "content": query_content}]

    api_key = os.environ["MISTRAL_API_KEY"]
    model = "mistral-large-latest"

    
    response = client.chat.complete(
        model = model,
        messages = messages,
        tools = tools,
        tool_choice = "any",
    )

    messages.append(response.choices[0].message)

    tool_call = response.choices[0].message.tool_calls[0]
    function_name = tool_call.function.name
    function_params = json.loads(tool_call.function.arguments)
    print("\nfunction_name: ", function_name, "\nfunction_params: ", function_params)

    function_result = names_to_functions[function_name](**function_params)

    messages.append({"role":"tool", "name":function_name, "content":function_result, "tool_call_id":tool_call.id})

    response = client.chat.complete(
        model = model,
        messages = messages
    )
    print("(ia) >>>", response.choices[0].message.content)
    messages.append({"role": "assistant", "content": response.choices[0].message.content})
    return messages

def chat_loop(messages=None):
    if messages == None:
        messages = []

    while True:
        new_query = input(">>>")
        if new_query == "end":
            break
        print()
        print("(user) >>>", new_query)
        messages.append({"role": "user", "content": new_query})
        response = client.chat.complete(
            model = model,
            messages = messages
        )
        print()
        print("(ia) >>>",response.choices[0].message.content)
        messages.append({"role": "assistant", "content": response.choices[0].message.content})
    


messages = pipeline_chat_assos("I want a yellow t-shirt")
chat_loop(messages)



function_name:  query_scrap_asos 
function_params:  {'query': 'yellow t-shirt'}
(ia) >>> Here are a couple of yellow t-shirt options from ASOS:

1. **Parlez cotton graphic print short sleeve t-shirt in yellow**
   - Brand: Parlez
   - Price: €47.19
   - Gender: Men
   - [View on ASOS](https://www.asos.com/parlez/parlez-cotton-graphic-print-short-sleeve-t-shirt-in-yellow/prd/205792521#colourWayId-205792522)

2. **Levi's varsity patch logo relaxed fit t-shirt in yellow**
   - Brand: Levi's
   - Price: €14.50
   - Gender: Men
   - [View on ASOS](https://www.asos.com/levis/levis-varsity-patch-logo-relaxed-fit-t-shirt-in-yellow/prd/206116564#colourWayId-206116585)

(user) >>> quel est la marque du premeir article ?

(ia) >>> La marque du premier article est **Parlez**.

(user) >>> quel est son prix ?

(ia) >>> Le prix du premier article, le t-shirt jaune de la marque Parlez, est de **€47.19**.
